<a href="https://colab.research.google.com/github/gyana-ranjan-dash/Safer-drinking-water/blob/main/water_probability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>